### CryoParticleSegment

In [ ]:
#%%bash
#git clone git+https://cyanazuki@github.com/cyanazuki/CryoParticleSegment.git
#cd CryoParticleSegment
#python setup.py install

## ⭐ Setup
You must run all codes under this category.

### ✅ Directory Settings

In [ ]:
# @title  { display-mode: "form" }

IMAGE_DIR = "/content/drive/MyDrive/research_xs/processed_micrographs_np" # @param {type:"string"}
LABEL_DIR = "/content/drive/MyDrive/research_xs/ground_truth_mask" # @param {type:"string"}
RESULT_DIR = "/content/drive/MyDrive/research_xs/final/fcn_resnet101/model_only" # @param {type:"string"}

In [ ]:
# @title  { display-mode: "form" }
# @markdown Detect whether using folder in Google Drive as **`RESULT DIR`**📁.

if "content" in IMAGE_DIR.split("/")[:3] or "content" in LABEL_DIR.split("/")[:3]:
  try:
    from google.colab import drive
    drive.mount('/content/drive')
    !rm -r /content/sample_data
    if "content" in IMAGE_DIR.split("/")[:3]:
      !cp -r {IMAGE_DIR} /content/image_dir
      IMAGE_DIR = "/content/image_dir"
    if "content" in LABEL_DIR.split("/")[:3]:
      !cp -r {LABEL_DIR} /content/label_dir
      LABEL_DIR = "/content/label_dir"
  except:
    pass

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
rm: cannot remove '/content/sample_data': No such file or directory


In [ ]:
# @title  { display-mode: "form" }
# @markdown Source code directory.
SRC_DIR = "/content/drive/MyDrive/research/src" # @param {type:"string"}

if True:
  !cp -r {SRC_DIR}/EM_project/*.py /content/
else:
  !cp {SRC_DIR}/EM_project/convcrf.py /content/convcrf.py
  !cp {SRC_DIR}/EM_project/dataset.py /content/dataset.py
  !cp {SRC_DIR}/EM_project/lr_scheduler.py /content/lr_scheduler.py
  !cp {SRC_DIR}/EM_project/metrics.py /content/metrics.py
  !cp {SRC_DIR}/EM_project/model.py /content/model.py
  !cp {SRC_DIR}/EM_project/trainer.py /content/trainer.py
  !cp {SRC_DIR}/EM_project/utils.py /content/utils.py

### ✅ Packages Handling

In [ ]:
# @title  { display-mode: "form" }
# @markdown Useful packages.

import os
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
# @title  { display-mode: "form" }
# @markdown User-defined packages.

from dataset import MicrographDataset, MicrographDatasetEvery
from model import create_model
from trainer import CryoEMTrainerWithScheduler, CryoEMEvaluator, tqdm_plugin_for_Trainer

## ⭐ Main

### ✅ Setting

In [ ]:
# @markdown Parameters.

NUM_CLASSES = 2
EPOCHS = 100
BATCH = 7
CROP_SIZE = (512, 512)
LR = 1e-3
RLR_PATIENCE = 3
ES_PATIENCE = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# @markdown Set seed.

random_state = 42
torch.manual_seed(random_state)
torch.cuda.manual_seed_all(random_state)

### ✅ Dataset

In [ ]:
train_dir = os.path.join(IMAGE_DIR, 'train')
train_filenames = np.loadtxt(f"{IMAGE_DIR}/train_filenames.txt", dtype=str)
train_dataset = MicrographDataset(image_dir=train_dir, label_dir=LABEL_DIR, filenames=train_filenames, crop_size=CROP_SIZE)

In [ ]:
val_dir = os.path.join(IMAGE_DIR, 'val')
val_filenames = np.loadtxt(f"{IMAGE_DIR}/val_filenames.txt", dtype=str)
val_dataset = MicrographDatasetEvery(image_dir=val_dir, label_dir=LABEL_DIR, filenames=val_filenames, crop_size=CROP_SIZE)
val_loader = DataLoader(val_dataset, batch_size=None, shuffle=False, pin_memory=True)

In [ ]:
test_dir = os.path.join(IMAGE_DIR, 'test')
test_filenames = np.loadtxt(f"{IMAGE_DIR}/test_filenames.txt", dtype=str)
test_dataset = MicrographDatasetEvery(image_dir=test_dir, label_dir=LABEL_DIR, filenames=test_filenames, crop_size=CROP_SIZE)
test_loader = DataLoader(test_dataset, batch_size=None, shuffle=False, pin_memory=True)

### ✅ Model

In [ ]:
backbone = torch.hub.load('pytorch/vision:v0.10.0', 'fcn_resnet101', num_classes=NUM_CLASSES)

backbone.backbone.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
backbone.classifier[3].p = 0.5

model = create_model(backbone)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 183MB/s]


In [ ]:
criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = ReduceLROnPlateau(optimizer, patience=RLR_PATIENCE, verbose=True)

## ⭐ Training

In [ ]:
Trainer = tqdm_plugin_for_Trainer(CryoEMTrainerWithScheduler)
trainer = Trainer(model, train_dataset, criterion, optimizer, device,
              num_classes = NUM_CLASSES,
              lr_scheduler=scheduler, patience=ES_PATIENCE)

trainer.train(EPOCHS, val_loader=val_loader, batch_size = BATCH,
              ckpt_dir = RESULT_DIR, random_state = random_state)

Epoch   1/100:


Training:   0%|          | 0/9 [00:00<?, ?it/s]

Training score:
  loss	: 0.5926


Validation:   0%|          | 0/9 [00:00<?, ?it/s]

Validation score:
  loss : 0.5851
  Average Precision : 0.0000
global correct: 75.35
average row correct: ['100.00', '0.00']
Recall: ['75.35', 'nan']
IoU: ['75.35', '0.00']
F1 Score: ['85.94', 'nan']
mean IoU: 37.68
Saving model at /content/drive/MyDrive/research_xs/final/fcn_resnet101/model_only/checkpoint1.pt
Loss improve to 0.5850959950023227.
Epoch   2/100:


...

Validation score:
  loss : 0.4378
  Average Precision : 0.1844
global correct: 91.41
average row correct: ['95.07', '80.24']
Recall: ['93.63', '84.18']
IoU: ['89.30', '69.73']
F1 Score: ['94.35', '82.16']
mean IoU: 79.51
No improvement for 19 epoch.
Epoch  94/100:


Training:   0%|          | 0/9 [00:00<?, ?it/s]

Training score:
  loss	: 0.4420


Validation:   0%|          | 0/9 [00:00<?, ?it/s]

Validation score:
  loss : 0.4385
  Average Precision : 0.1826
global correct: 91.33
average row correct: ['95.22', '79.43']
Recall: ['93.40', '84.46']
IoU: ['89.22', '69.30']
F1 Score: ['94.30', '81.87']
mean IoU: 79.26
Early stopping


In [ ]:
import pickle

trainer_result = {
    '_results': trainer._results,
    'best_epoch': trainer.best_epoch,
    'best_loss': trainer.best_loss,
    'loss': trainer.loss,
}

with open(f'{RESULT_DIR}/trainer_result.pickle', 'wb') as f:
    pickle.dump(trainer_result, f)

## ⭐ Evaluate

### ⏭ Test score for all saved checkpoint.

In [ ]:
from torchvision.utils import save_image
from dataset import reconstruct_patched

In [ ]:
checkpoint_paths = [path for path in os.listdir(RESULT_DIR) if '.pt' in path]
for checkpoint_path in checkpoint_paths:
  state_dict_path = f"{RESULT_DIR}/{checkpoint_path}"
  state_dict = torch.load(state_dict_path, map_location=torch.device(device))
  model.load_state_dict(state_dict, strict=False)
  model.eval()
  print("\nLoad model at: ", state_dict_path)
  trainer = CryoEMTrainerWithScheduler(model, train_dataset, criterion, optimizer, device,
                num_classes=NUM_CLASSES,
                lr_scheduler=scheduler, patience=ES_PATIENCE)
  result = trainer.evaluate(test_loader)
  model.to(device)
  model.eval()
  with torch.no_grad():
    idx, (test_image, _, grid, _) = next(enumerate(test_dataset))
    inputs = test_image.to(device)
    outputs = model(inputs)['out']
    preds = outputs.argmax(dim=1).cpu().detach()
    filename = f"{os.path.splitext(checkpoint_path)[0]}.png"
    pred_path = os.path.join(RESULT_DIR, "Each_ckpt", filename)
    save_image(reconstruct_patched(preds, grid).float(), pred_path)

### ⏭ Load previous

In [ ]:
# @markdown Set show_checkpoint to True to check every checkpoint.
# @markdown If checkpoint_filename is "" (empty), the last checkpoint will be used.
show_checkpoint = False # @param {type:"boolean"}
checkpoint_filename = "" # @param {type:"string"}
if True:
  checkpoint_paths = [path for path in os.listdir(RESULT_DIR) if '.pt' in path]
  if show_checkpoint:
    print(checkpoint_paths)
  if str(checkpoint_filename) == "":
    checkpoint_filename = checkpoint_paths[-1]
  state_dict_path = f"{RESULT_DIR}/{checkpoint_filename}"
  state_dict = torch.load(state_dict_path, map_location=torch.device(device))
  model.load_state_dict(state_dict, strict=False)
  model.eval()
  print("Load model at: ", state_dict_path)
  try:
    # Run if changing trainer.
    import pickle
    # !rm __pycache__/*
    # !cp /content/drive/MyDrive/research/src/EM_project/trainer.py /content/trainer.py
    # from trainer import CryoEMTrainerWithScheduler
    Trainer = tqdm_plugin_for_Trainer(CryoEMTrainerWithScheduler)
    trainer = Trainer(model, train_dataset, val_dataset, criterion, optimizer, device,
                  num_classes = NUM_CLASSES,
                  lr_scheduler=scheduler, patience=ES_PATIENCE)
    with open(f'{RESULT_DIR}/trainer_result.pickle', 'rb') as f:
      trainer_result = pickle.load(f)
    for key in trainer_result:
      setattr(trainer, key, trainer_result[key])
    # for var in dir(trainer):
    #   if '__' not in var:
    #     print(var)
    #     print(f'var:\n  ', getattr(trainer, var))
  except:
    pass

Load model at:  /content/drive/MyDrive/research_xs/final/fcn_resnet101/model_only/checkpoint74.pt


### ✅ Testing

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
evaluator = CryoEMEvaluator(model=model, device=device, metrics=['iou'], num_classes=NUM_CLASSES)
print("FCN validation result:")
result = evaluator.evaluate(loader=val_loader)
print("FCN test result:")
result = evaluator.evaluate(loader=test_loader)

FCN validation result:
  Average Precision : 0.1859
global correct: 91.44
average row correct: ['94.86', '80.98']
Recall: ['93.85', '83.75']
IoU: ['89.30', '69.99']
F1 Score: ['94.35', '82.34']
mean IoU: 79.65
FCN test result:
  Average Precision : 0.1580
global correct: 90.29
average row correct: ['94.27', '76.50']
Recall: ['93.30', '79.37']
IoU: ['88.29', '63.81']
F1 Score: ['93.78', '77.91']
mean IoU: 76.05


In [ ]:
from torchvision.utils import save_image
from dataset import reconstruct_patched

!mkdir {RESULT_DIR}/test_image
model.eval()
with torch.no_grad():
  for idx, (test_image, _, grid, _) in enumerate(test_dataset):
    inputs = test_image.to(device)
    outputs = model(inputs)['out']
    preds = outputs.argmax(dim=1).cpu().detach()
    filename = f"{os.path.splitext(test_dataset.filenames[idx])[0]}.png"
    pred_path = os.path.join(RESULT_DIR, "test_image", filename)
    save_image(reconstruct_patched(preds, grid).float(), pred_path)